In [1]:
import pandas as pd
import re
from datetime import datetime

# Cartella dove si trovano i file da analizzare
cartella_lavoro = r"C:\Users\Antonio Maurizi\Desktop\PROGETTO ESAME EPICODE"

# Elenco dei file CSV da unire (ognuno rappresenta un periodo diverso) e creazione di una LISTA
reportistiche_copertura = [
    "Reportistica_250930_Comuni.csv",
    "Reportistica_250630_Comuni.csv",
    "Reportistica_250331_Comuni.csv",
    "Reportistica_241231_Comuni.csv",
    "Reportistica_240930_Comuni.csv",
    "Reportistica_240630_Comuni.csv",
    "Reportistica_240331_Comuni.csv",
    "Reportistica_231231_Comuni.csv",
    "Reportistica_230930_Comuni.csv",
    "Reportistica_230630_Comuni.csv"
]

# creazione di un dizianario utile alla causa in quanto non si anno date di riferimento se non nei nomi dei csv
# nello specifico (es: "24" = 2024, "03" = Q1)


DATADESC = {
    "25": "2025",
    "24": "2024",
    "23": "2023",
    "22": "2022",
    "12": "Q4",
    "09": "Q3",
    "06": "Q2",
    "03": "Q1"
}

# creazione dizionario
# Nei file la stessa informazione può avere nomi diversi, qui uniformiamo i nomi delle colonne per usarle più facilmente dopo.
colonne_rettifica = {
    "Punti dichiarati": "Punti terminazione dichiarati",
    "Punti geograficamente distinti (passo 5m)": "Punti terminazione geograficamente distinti (passo 5m)",
    "Indirizzi postali distinti (se fornito/disponibile)": "Indirizzi postali distinti (se forniti/disponibili)",
    "Celle 20m x 20m raggiunte FTTH": "Celle 20m x 20m raggiunte FTTH",
    "Punti dichiarati FTTH": "Punti terminazione dichiarati FTTH",
    "Punti Geograficamente Distinti (passo 5m) FTTH": "Punti terminazione geograficamente distinti (passo 5m) FTTH"
}

# creazione della lista o contenitore e ciclo for su tutti i file csv con l'utilizzo del ciclo for ed estrazione numerica
# per ricavare la data di riferimento e la pulizia delle varie colonne
# altresì con replace e regex ho potuto correggere i caratteri non conformi 
Dataset_FTTH_FTTC_Italia = []
for reportistica in reportistiche_copertura:
    data = re.findall(r'\d+', reportistica)
    data = "".join(data)
    
    reportistica = pd.read_csv(f"{cartella_lavoro}\\{reportistica}", sep=';', encoding='latin-1')
    reportistica.columns = (
        reportistica.columns
        .str.replace('\n', ' ', regex=True)
        .str.replace(r'\\', ' ', regex=True)
        .str.replace('ï»¿', '', regex=False)
        .str.strip()
    )
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã¬', 'ì', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã²', 'ò', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã¨', 'è', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã¹', 'ù', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã´', 'ô', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã§', 'ç', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã¢', 'â', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã®', 'î', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã¤', 'ä', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ãº', 'ú', regex=False)
    reportistica['Comune'] = reportistica['Comune'].str.replace('Ã', 'à', regex=False)
    reportistica = reportistica.loc[:, ~reportistica.columns.str.startswith("Unnamed")]

    
    reportistica = reportistica.dropna(how='all')

    # Ricavo la data dal nome file con creazione data e l'utilizzo del dizionario per crearne una di riferimento
    anno_code = data[:2]
    mese_code = data[2:4]
    giorno_code = data[4:6]
    DataRiferimento = datetime(int("20" + anno_code), int(mese_code), int(giorno_code))
    reportistica["DataRiferimento"] = DataRiferimento

    anno = DATADESC[data[:2]]
    trimestre = DATADESC[data[2:4]]
    reportistica["anno"] = anno
    reportistica["trimestre"] = trimestre
    reportistica = reportistica.rename(columns=colonne_rettifica)

    Dataset_FTTH_FTTC_Italia.append(reportistica)
    
Fiber_Vision_Actual = pd.concat(Dataset_FTTH_FTTC_Italia, ignore_index=True)

# Formatto i numeri per togliere ".0" se sono numeri interi
for col in Fiber_Vision_Actual.columns:
    if Fiber_Vision_Actual[col].dtype in ['float64', 'float32']:
        Fiber_Vision_Actual[col] = Fiber_Vision_Actual[col].apply(
            lambda x: f"{int(x)}" if pd.notnull(x) and float(x).is_integer() else x
        )

display(Fiber_Vision_Actual)


,Regione,Provincia,Comune,pro_com,Celle 20m x 20m raggiunte,Punti terminazione dichiarati,Punti terminazione geograficamente distinti (passo 5m),Indirizzi postali distinti (se forniti/disponibili),Celle 20m x 20m raggiunte FTTH,Punti terminazione dichiarati FTTH,Punti terminazione geograficamente distinti (passo 5m) FTTH,Indirizzi postali distinti (se forniti/disponibili) FTTH,Celle 20m x 20m raggiunte FTTC,Famiglie Residenti (calcolato),Famiglie raggiunte FTTH (calcolato),DataRiferimento,anno,trimestre
0,Abruzzo,Chieti,Altino,69001,1137,8538,2025,5183,143,295,237,966,934,1092,339,2025-09-30,2025,Q3
1,Abruzzo,Chieti,Archi,69002,743,6137,1157,3732,0,0,0,NaN,511,865,2,2025-09-30,2025,Q3
2,Abruzzo,Chieti,Ari,69003,317,1355,602,1457,119,370,231,726,1,441,320,2025-09-30,2025,Q3
3,Abruzzo,Chieti,Arielli,69004,368,1763,708,5803,124,519,305,2760,1,421,298,2025-09-30,2025,Q3
4,Abruzzo,Chieti,Atessa,69005,3389,26689,6063,15385,1158,4496,2765,7966,2112,3854,2798,2025-09-30,2025,Q3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78758,Veneto,Vicenza,Villaverla,24118,2452,10249,4499,4719,1437,2191,2145,78,NaN,NaN,NaN,2023-06-30,2023,Q2
78759,Veneto,Vicenza,Zanè,24119,2866,22892,5606,9222,1600,2190,2154,401,NaN,NaN,NaN,2023-06-30,2023,Q2
78760,Veneto,Vicenza,Zermeghedo,24120,565,1477,934,1241,324,444,440,35,NaN,NaN,NaN,2023-06-30,2023,Q2
78761,Veneto,Vicenza,Zovencedo,24121,334,1541,415,655,0,0,0,0,NaN,NaN,NaN,2023-06-30,2023,Q2
